# 파이썬 데코레이터.

파이썬에서 데코레이터는 코드의 수정 없이 기존 함수나 메소드에 새로운 기능을 추가하는 매커니즘이며, 주로 코드의 재사용성을 증가시키기 위해 사용됩니다.<br>
기본적으로, 데코레이터는 다른 함수를 인자로 받아 그 함수를 수정하거나 확장한 후, 수정된 함수를 반환하는 함수입니다.<br>

In [23]:
# 기본 예제.

def simple_decorator(func):
    print(123)
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper
    

@simple_decorator # 데코레이터는 말그대로 장식(꾸미다)이다. say_hello() 함수를 꾸밈.
def say_hello():
    print("Hello!")

say_hello()

# 파이썬에서 @ 표시로 시작되는 데코레이터가 사용되면, 그 아래에 정의된 함수는 데코레이터 함수의 인자로 즉시 전달됩니다. 
# 따라서 결국 say_hello()함수는 wrapper 함수를 참조하게 된다. 이 말이 무슨 뜻인지 자세히 알아보자.

# say_hello 함수에 데코레이터(simpe_decorator)를 적용해보자.
# @simple_decorator
# def say_hello():
#     print("Hello!")

# 위 코드는 실제로는 아래와 같이 해석된다. (이는 파이썬에 내장되어있는 기본적인 데코레이터 문법 설계이다.)
# say_hello = simple_decorator(say_hello)

# 결론.
# 데코레이터를 정의(사용)하면 파이썬 내부적으로 설계된 문법에 의하여 say_hello = simple_decorator(say_hello) 이와 같이 작동된다. (simple_decorator(say_hello)는 wrapper함수를 반환한다)
# 따라서 say_hello = wrapper이기 때문에 say_hello 함수가 wrapper함수를 참조한다. 그래서 say_hello() 호출시 wrapper()함수가 호출되는 것이다.

# 참고로 print(123)이 먼저 출력된 이유도 위와 같은 이유이기 때문이다. say_hello = simple_decorator(say_hello) 이 과정에서 print()문이 실행되기때문에.

123
Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In [32]:
# 인자를 받는 함수에 대한 데코레이터.
# 만약 데코레이트하려는 함수가 인자를 받는다면 어떻게 해야 할까요? `*args`와 `**kwargs`를 사용하여 모든 종류의 인자를 받아들일 수 있는 데코레이터를 작성할 수 있습니다.

def general_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before the function call")
        result = func(*args, **kwargs)
        print("After the function call")
        return result
    return wrapper

@general_decorator # add = general_decorator(add). 참고로, 데코레이터는 함수 객체 자체를 인자로 받는다, 함수의 실행 결과를 받는 것이 아니다. 따라서 say_hello()가 아닌 say_hello가 인자로 들어감.
def add(a, b):
    return a + b

print(add(3, 4)) # add = general_decorator(add)이 식으로 인해 wrapper(3, 4)를 호출하는 것과 같음. 
# 또한 print()문으로 wrapper(3, 4)의 return 값을 출력했음.

Before the function call
After the function call
7


In [31]:
# 데코레이터에 인자 전달하기,
# 데코레이터 자체도 인자를 받을 수 있습니다. 이를 위해서는 `데코레이터를 반환하는 함수`를 추가로 작성해야 합니다.

def repeat(num_times): # 데코레이터를 반환하기위해 함수를 추가로 작성.
    def decorator_repeat(func): # 데코레이터.
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs) # greet() 함수에 반환값이 없음으로 result변수는 없어도 되지만 예제이기 때문에 일반적인 경우를 가정하여 적음.
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3) # greet = repeat(num_times=3)(greet). => 이러한 func()()형태의 함수를 고차 함수라한다. (func()가 호출되면 함수를 반환하고, 그 반환된 함수가 다시 호출되는 형태를 나타냅니다.)
def greet(name):
    print(f"Hello {name}")

greet("Alice") # greet() 함수에 반환값이 없음으로 print(greet("Alice"))할 필요 없음.

# 고차 함수의 예.   
print()
def outer_function():
    def inner_function():
        return "Inner function called!"
    return inner_function

result = outer_function()()  # outer_function을 호출한 후, 그 결과로 반환된 inner_function을 호출합니다.
print(result)  # 출력: Inner function called!

Hello Alice
Hello Alice
Hello Alice

Inner function called!


# SocketIO 클래스 사용을 이해하기 위해 알아야할 기본적인 이벤트 리스너 패턴.

아래는 이벤트 리스너를 간단하게 흉내내 보는 예제입니다:

```python
class EventSystem:
    def __init__(self):
        self.listeners = {}
    # 위의 데코레이터들 중에 데코레이터에 인자 전달하기 부분이다.
    def on(self, event_name): # 데코레이터를 반환하기 위해 추가로 작성한 함수.
        def decorator(func): # 데코레이터.
            self.listeners.setdefault(event_name, []).append(func)
            return func # return func: 함수 객체 자체를 반환. return func(): 함수를 호출하고 그 결과를 반환.
        return decorator

    def emit(self, event_name, *args, **kwargs):
        if event_name in self.listeners:
            for listener in self.listeners[event_name]:
                listener(*args, **kwargs)

events = EventSystem()

@events.on('data_received') # events.on() 이라는 뜻은 on메서드를 호출한다는 뜻. 즉, events.on('data_received')는 on 메서드를 호출하며, 그 결과로 반환된 데코레이터 함수가 그 아래에 정의된 함수를 장식(데코레이트)하게 됩니다.
def print_data(data):       # print_data = events.on('data_received')(print_data).
    print(f"Data received: {data}")

@events.on('data_received') # store_data = events.on('data_received')(store_data).
def store_data(data):
    print(f"Storing data: {data}")

@events.on('error')
def error_handler(message):
    print(f"Error occurred: {message}")

events.emit('data_received', "Hello, world!")
events.emit('error', "Something went wrong!")
```

이 예제에서 `EventSystem` 클래스는 간단한 이벤트 시스템을 제공합니다. `on` 메서드는 데코레이터를 반환하여 특정 이벤트 이름에 함수(리스너)를 연결합니다. `emit` 메서드는 특정 이벤트를 발생시켜 해당 이벤트에 연결된 모든 함수를 호출합니다.

이 예제를 실행하면 다음과 같은 출력을 얻습니다:

```
Data received: Hello, world!
Storing data: Hello, world!
Error occurred: Something went wrong!
```

이와 같이 파이썬의 기본 문법만을 사용하여도 다양한 데코레이터와 패턴을 구현할 수 있습니다.

### "위 코드 중에 `self.listeners.setdefault(event_name, []).append(func)` 이부분에 대해 자세히 알아보자."

1. **`self.listeners`**: 
   - `EventSystem` 클래스의 인스턴스 변수입니다. 이 딕셔너리는 이벤트 이름을 키로, 해당 이벤트에 대한 리스너 함수들의 목록을 값으로 가집니다.

2. **`setdefault` 메서드**:
   - `setdefault`는 딕셔너리의 메서드로, 주어진 키에 대한 값이 딕셔너리에 이미 있으면 그 값을 반환하고, 없으면 주어진 기본값을 딕셔너리에 설정하고 그 값을 반환합니다.
   - 여기서 사용한 `setdefault(event_name, [])`는 다음과 같은 동작을 합니다:
     - `event_name` 키가 `self.listeners`에 존재하면 해당 키의 값을 반환합니다.
     - `event_name` 키가 `self.listeners`에 존재하지 않으면 빈 리스트 (`[]`)를 그 키의 값으로 설정하고 그 빈 리스트를 반환합니다.
   
3. **`.append(func)`**:
   - `setdefault` 메서드의 반환값은 리스트입니다. (이벤트 이름에 대한 리스너 함수들의 목록)
   - `.append(func)`는 이 리스트에 `func` 함수를 추가합니다. 여기서 `func`는 `on` 메서드의 데코레이터가 장식하는 함수입니다.

코드를 단계별로 분해해서 보면 다음과 같습니다:

1. `event_name`이 `self.listeners` 딕셔너리에 있으면 해당 키의 값을 가져옵니다. 없으면, 빈 리스트를 해당 키의 값으로 설정하고 그 리스트를 가져옵니다.
2. 이 리스트 (`self.listeners[event_name]`)에 `func` 함수를 추가합니다. `참고로 함수도 객체임으로 리스트에 함수도 추가할 수 있다.` (list = [func1, func2])

결론적으로, 이 로직은 주어진 이벤트 이름에 대한 리스너 함수 목록을 관리하며, 새로운 리스너 함수가 데코레이터를 통해 추가될 때마다 해당 함수를 그 이벤트의 리스너 목록에 추가합니다.